In [ ]:
# spring.neo4j.uri=neo4j+s://ed887860.databases.neo4j.io:7687
# spring.neo4j.authentication.username=neo4j
# spring.neo4j.authentication.password=yulO-mLLrt72cJ39FI12Lo-Lr6wtv6qx2oIzUNDl5Zo
# spring.data.neo4j.database = neo4

In [37]:
# Import py2neo and connect to Neo4j
from py2neo import Graph

# just an example, replace with credentials for your own Neo4j instance
# graph = Graph(bolt=False, host="ed887860.databases.neo4j.io", http_port=7687, user='neo4j', password='yulO-mLLrt72cJ39FI12Lo-Lr6wtv6qx2oIzUNDl5Zo')
graph = Graph('neo4j+s://ed887860.databases.neo4j.io:7687', auth=('neo4j','yulO-mLLrt72cJ39FI12Lo-Lr6wtv6qx2oIzUNDl5Zo'))

In [3]:
# Hello world, sanity check
graph.run("MATCH (a) RETURN COUNT(a) AS numberOfNodes").evaluate()

170

In [4]:
result = graph.run('''
MATCH (domain:Domain)<-[:IS_LINKED_WITH]->(tag:Tag)<-[:IS_LINKED_WITH]->(sug:Suggestion)
WHERE domain.name CONTAINS "Automotive" AND tag.name CONTAINS "used"
RETURN sug.name
LIMIT 10
''')

for row in result:
    print(row)

'Canadian Used Vehicles - Toyota'
'Canadian GM Inventory'


In [5]:
result = graph.run('''
MATCH (domain:Domain)<-[:IS_LINKED_WITH]->(tag:Tag)<-[:IS_LINKED_WITH]->(sug:Suggestion)
WHERE domain.name CONTAINS "Automotive" AND tag.name CONTAINS "used"
RETURN DISTINCT sug.name
LIMIT 10
''')

In [6]:
arr = result.to_ndarray()
arr[0]

array(['Canadian Used Vehicles - Toyota'], dtype='<U31')

In [81]:
# domain = "Automotive"
# tags = ["used","toyota"]

domain = "Automotion"
tags = ["test1","test2"]

quary = '''
MATCH (domain:Domain)<-[:IS_LINKED_WITH]->(tag:Tag)<-[:IS_LINKED_WITH]->(sug:Suggestion)
WHERE domain.name CONTAINS "'''+domain+'''"
WITH sug,COUNT(sug) AS suggestions
ORDER BY suggestions DESC
RETURN DISTINCT sug.name
LIMIT 10
'''

quary

'\nMATCH (domain:Domain)<-[:IS_LINKED_WITH]->(tag:Tag)<-[:IS_LINKED_WITH]->(sug:Suggestion)\nWHERE domain.name CONTAINS "Automotion"\nWITH sug,COUNT(sug) AS suggestions\nORDER BY suggestions DESC\nRETURN DISTINCT sug.name\nLIMIT 10\n'

In [82]:
tag = tags[0]
tag_qu = ' AND tag.name CONTAINS "'+ tag +'"'
substr = "WITH sug"
idx = quary.index(substr)
quary = quary[:idx] + tag_qu + quary[idx:]
for tag in tags[1:]:
    tag_qu = ' OR tag.name CONTAINS "'+ tag+'"'
    substr = "WITH sug"
    idx = quary.index(substr)
    quary = quary[:idx] + tag_qu + quary[idx:]

quary

'\nMATCH (domain:Domain)<-[:IS_LINKED_WITH]->(tag:Tag)<-[:IS_LINKED_WITH]->(sug:Suggestion)\nWHERE domain.name CONTAINS "Automotion"\n AND tag.name CONTAINS "test1" OR tag.name CONTAINS "test2"WITH sug,COUNT(sug) AS suggestions\nORDER BY suggestions DESC\nRETURN DISTINCT sug.name\nLIMIT 10\n'

In [83]:
result_multi = graph.run(quary)
result_multi

(No data)

In [89]:
if result_multi:
    print('ok')

In [84]:
import pandas as pd

In [85]:
# result_data = result_multi.to_data_frame()
# j_li = list(result_data['sug.name'])
# j_li

In [86]:
result_pd = pd.DataFrame(result_multi.to_data_frame())

In [87]:
result_pd

""


In [90]:
if result_pd.empty:
    print('ok')

ok


In [71]:
j_li = list(result_pd['sug.name'])
j_li

['Canadian Used Vehicles - Toyota', 'Canadian GM Inventory']

In [75]:
result_fi = {'suggestion':list(result_pd['sug.name'])}
result_fi

{'suggestion': ['Canadian Used Vehicles - Toyota', 'Canadian GM Inventory']}

In [ ]:
# my_str = "thisissometextthatiwrote"
# substr = "text"
# inserttxt = "XX"

# idx = my_str.index(substr)
# my_str = my_str[:idx] + inserttxt + my_str[idx:]

In [ ]:
result_multi

In [ ]:
result_multi = graph.run('''
MATCH (domain:Domain)<-[:IS_LINKED_WITH]->(tag:Tag)<-[:IS_LINKED_WITH]->(sug:Suggestion)
WHERE domain.name CONTAINS "Automotive" AND tag.name CONTAINS "used"
RETURN DISTINCT sug.name
LIMIT 10
''')

In [9]:
tag_qu

'AND tag.name CONTAINS"toyota"'

In [4]:
from pydantic import BaseModel,constr

In [30]:
class suggestions(BaseModel):
    domain:str
    tags: list[constr(max_length=255)]

In [31]:
item_py = suggestions(
    domain = "auto",
    tags = ["test1","test2"]
    )

In [32]:
item_py.domain

'auto'

In [33]:
item_py.tags

['test1', 'test2']

In [23]:
item1 = '{"domain":"auto", "tag":["test1","test2"]}'

In [25]:
import json
item = json.loads(item1)
domain = item['domain']
domain

'auto'

In [ ]:
def recommendatio(item:suggestions):